In [ ]:
%%writefile v4.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <cuda_fp16.h>
typedef uint16_t fp16;

#define max(a,b) ((a) > (b) ? (a) : (b))
#define INPUT_SIZE 784
#define HIDDEN_SIZE 128
#define OUTPUT_SIZE 10
#define LEARNING_RATE 0.01
#define EPOCHS 3
#define BATCH_SIZE 64
#define NUM_CLASSES 10  // Digits 0-9
#define cudaCheckError() {                                          \
    cudaError_t e=cudaGetLastError();                                \
    if(e!=cudaSuccess) {                                             \
        printf("CUDA Error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(e)); \
        exit(EXIT_FAILURE);                                          \
    }                                                                \
}


// Timer function
double get_time(clock_t start) {
    return (double)(clock() - start) / CLOCKS_PER_SEC;
}

// Neural network structure for host
typedef struct {
    half** W1;  // Weight matrix for input to hidden layer
    half** W2;  // Weight matrix for hidden to output layer
    half* b1;   // Bias for hidden layer
    half* b2;   // Bias for output layer
} NeuralNetwork;

// Neural network structure for device
typedef struct {
    half* W1_flat;  // Flattened weight matrix for input to hidden layer
    half* W2_flat;  // Flattened weight matrix for hidden to output layer
    half* b1;       // Bias for hidden layer
    half* b2;       // Bias for output layer
} NeuralNetworkDevice;


// Allocate memory for a matrix of type half
half** allocateMatrix(int rows, int cols) {
    half** mat = (half**)malloc(rows * sizeof(half*));
    for (int i = 0; i < rows; i++) {
        mat[i] = (half*)malloc(cols * sizeof(half));
    }
    return mat;
}

// Free allocated matrix memory
void freeMatrix(half** mat, int rows) {
    for (int i = 0; i < rows; i++) {
        free(mat[i]);
    }
    free(mat);
}


// Initialize neural network with random values
NeuralNetwork* createNetwork() {
    NeuralNetwork* net = (NeuralNetwork*)malloc(sizeof(NeuralNetwork));
    net->W1 = allocateMatrix(HIDDEN_SIZE, INPUT_SIZE);
    net->W2 = allocateMatrix(OUTPUT_SIZE, HIDDEN_SIZE);
    net->b1 = (half*)calloc(HIDDEN_SIZE, sizeof(half));
    net->b2 = (half*)calloc(OUTPUT_SIZE, sizeof(half));

    srand(time(NULL));
    for (int i = 0; i < HIDDEN_SIZE; i++)
        for (int j = 0; j < INPUT_SIZE; j++)
            net->W1[i][j] = __float2half(((float)rand() / RAND_MAX) * 0.01);

    for (int i = 0; i < OUTPUT_SIZE; i++)
        for (int j = 0; j < HIDDEN_SIZE; j++)
            net->W2[i][j] = __float2half(((float)rand() / RAND_MAX) * 0.01);

    return net;
}


// Forward pass
__device__ half relu(half x) {
    return __hgt(x, __float2half(0.0f)) ? x : __float2half(0.0f);
}

// ---------- Forward kernell ----------

__global__ void forward_kernell(NeuralNetwork* net, double* input, double* hidden, double* output) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Hidden layer
    if (idx < HIDDEN_SIZE) {
        double sum = net->b1[idx];
        for (int j = 0; j < INPUT_SIZE; j++) {
            sum += net->W1[idx][j] * input[j];
        }
        hidden[idx] = relu(sum);
    }

    __syncthreads();  // Ensure all hidden activations are ready

    // Output layer
    if (idx < OUTPUT_SIZE) {
        double sum = net->b2[idx];
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            sum += net->W2[idx][j] * hidden[j];
        }
        output[idx] = sum;
        printf("%f",output[idx]);

    }
}
// ---------- Forward kernel ----------

__global__ void forward_kernel(NeuralNetworkDevice* net, double* input, double* hidden, double* output) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Hidden layer
    if (idx < HIDDEN_SIZE) {
        double sum = net->b1[idx];
        for (int j = 0; j < INPUT_SIZE; j++) {
            sum += net->W1_flat[idx*INPUT_SIZE+j] * input[j];
        }
        hidden[idx] = relu(sum);
    }

    __syncthreads();  // Ensure all hidden activations are ready

    // Output layer
    if (idx < OUTPUT_SIZE) {
        double sum = net->b2[idx];
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            sum += net->W2_flat[idx*HIDDEN_SIZE+j] * hidden[j];
        }
        output[idx] = sum;
        printf("%f",output[idx]);

    }
}

// ---------- Softmax kernel ----------

__global__ void softmax_kernel(double* x, int size, double* sum_out) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Step 1: Exponentiate
    if (idx < size) {
        x[idx] = exp(x[idx]);
    }

    __syncthreads();

    // Step 2: Compute sum using thread 0
    if (idx == 0) {
        double sum = 0.0;
        for (int i = 0; i < size; i++) {
            sum += x[i];
        }
        *sum_out = sum;
    }

    __syncthreads();

    // Step 3: Normalize
    double sum_val = *sum_out;
    if (idx < size && sum_val != 0.0) {
        x[idx] /= sum_val;
        printf("sum %f",sum_val );
    }
}

// Backpropagation with FP16
void backward(NeuralNetworkDevice* net, half* input, half* hidden, half* output, half* target) {
    __shared__ half d_output[OUTPUT_SIZE];
    __shared__ half d_hidden[HIDDEN_SIZE];

    // Compute output layer gradient
    for (int i = 0; i < OUTPUT_SIZE; i++) {
        d_output[i] = __hsub(output[i], target[i]);
    }

    __syncthreads();

    // Compute hidden layer gradient
    for (int i = 0; i < HIDDEN_SIZE; i++) {
        d_hidden[i] = __float2half(0.0f);
        for (int j = 0; j < OUTPUT_SIZE; j++) {
            d_hidden[i] = __hadd(d_hidden[i], __hmul(net->W2_flat[j * HIDDEN_SIZE + i], d_output[j]));
        }
        d_hidden[i] = __hmul(d_hidden[i], __hgt(hidden[i], __float2half(0.0f)));
    }

    __syncthreads();

    // Update weights (gradient descent)
    for (int i = 0; i < OUTPUT_SIZE; i++) {
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            net->W2_flat[i * HIDDEN_SIZE + j] = __hsub(net->W2_flat[i * HIDDEN_SIZE + j], __hmul(d_output[i], hidden[j]));
        }
    }

    for (int i = 0; i < HIDDEN_SIZE; i++) {
        for (int j = 0; j < INPUT_SIZE; j++) {
            net->W1_flat[i * INPUT_SIZE + j] = __hsub(net->W1_flat[i * INPUT_SIZE + j], __hmul(d_hidden[i], input[j]));
        }
    }

    for (int i = 0; i < OUTPUT_SIZE; i++) {
        net->b2[i] = __hsub(net->b2[i], d_output[i]);
    }

    for (int i = 0; i < HIDDEN_SIZE; i++) {
        net->b1[i] = __hsub(net->b1[i], d_hidden[i]);
    }
}


// Copy data from host to device
void copyHostToDevice(NeuralNetwork* host, NeuralNetworkDevice* device) {
    int size_W1 = INPUT_SIZE * HIDDEN_SIZE * sizeof(half);
    int size_W2 = HIDDEN_SIZE * OUTPUT_SIZE * sizeof(half);
    int size_b1 = HIDDEN_SIZE * sizeof(half);
    int size_b2 = OUTPUT_SIZE * sizeof(half);

    cudaMalloc((void**)&device->W1_flat, size_W1);
    cudaMalloc((void**)&device->W2_flat, size_W2);
    cudaMalloc((void**)&device->b1, size_b1);
    cudaMalloc((void**)&device->b2, size_b2);

    half* W1_flat_host = (half*)malloc(size_W1);
    half* W2_flat_host = (half*)malloc(size_W2);
    for (int i = 0; i < HIDDEN_SIZE; i++) {
        for (int j = 0; j < INPUT_SIZE; j++) {
            W1_flat_host[i * INPUT_SIZE + j] = net->W1[i][j];
        }
    }
    for (int i = 0; i < OUTPUT_SIZE; i++) {
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            W2_flat_host[i * HIDDEN_SIZE + j] = net->W2[i][j];
        }
    }

    cudaMemcpy(device->W1_flat, W1_flat_host, size_W1, cudaMemcpyHostToDevice);
    cudaMemcpy(device->W2_flat, W2_flat_host, size_W2, cudaMemcpyHostToDevice);
    cudaMemcpy(device->b1, host->b1, size_b1, cudaMemcpyHostToDevice);
    cudaMemcpy(device->b2, host->b2, size_b2, cudaMemcpyHostToDevice);

    free(W1_flat_host);
    free(W2_flat_host);
}

// Copy data from device to host
void copyDeviceToHost(NeuralNetworkDevice* device, NeuralNetwork* host) {
    int size_W1 = INPUT_SIZE * HIDDEN_SIZE * sizeof(half);
    int size_W2 = HIDDEN_SIZE * OUTPUT_SIZE * sizeof(half);
    int size_b1 = HIDDEN_SIZE * sizeof(half);
    int size_b2 = OUTPUT_SIZE * sizeof(half);

    half* W1_flat_host = (half*)malloc(size_W1);
    half* W2_flat_host = (half*)malloc(size_W2);

    cudaMemcpy(W1_flat_host, device->W1_flat, size_W1, cudaMemcpyDeviceToHost);
    cudaMemcpy(W2_flat_host, device->W2_flat, size_W2, cudaMemcpyDeviceToHost);
    cudaMemcpy(host->b1, device->b1, size_b1, cudaMemcpyDeviceToHost);
    cudaMemcpy(host->b2, device->b2, size_b2, cudaMemcpyDeviceToHost);

    for (int i = 0; i < INPUT_SIZE; i++) {
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            host->W1[i][j] = W1_flat_host[i * HIDDEN_SIZE + j];
        }
    }
    for (int i = 0; i < HIDDEN_SIZE; i++) {
        for (int j = 0; j < OUTPUT_SIZE; j++) {
            host->W2[i][j] = W2_flat_host[i * OUTPUT_SIZE + j];
        }
    }

    free(W1_flat_host);
    free(W2_flat_host);
}


// Free device memory
void freeDeviceNeuralNetwork(NeuralNetworkDevice** device) {
    cudaFree((*device)->W1_flat);
    cudaFree((*device)->W2_flat);
    cudaFree((*device)->b1);
    cudaFree((*device)->b2);
}


__global__ void backwardKernel(
    __half* W1_flat, __half* W2_flat, __half* b1, __half* b2,
    __half* input, __half* hidden, __half* output, __half* target
) {
    __shared__ __half d_output[OUTPUT_SIZE];
    __shared__ __half d_hidden[HIDDEN_SIZE];

    int tid = threadIdx.x;

    if (tid < OUTPUT_SIZE) {
        d_output[tid] = __hsub(output[tid], target[tid]);
    }
    __syncthreads();

    if (tid < HIDDEN_SIZE) {
        d_hidden[tid] = __float2half(0.0f);
        for (int j = 0; j < OUTPUT_SIZE; j++) {
            d_hidden[tid] = __hfma(W2_flat[j * HIDDEN_SIZE + tid], d_output[j], d_hidden[tid]);
        }
        d_hidden[tid] = __hmul(d_hidden[tid], __hgt(hidden[tid], __float2half(0.0f)));
    }
    __syncthreads();

    // Update W2
    if (tid < OUTPUT_SIZE) {
        for (int j = 0; j < HIDDEN_SIZE; j++) {
            W2_flat[tid * HIDDEN_SIZE + j] = __hsub(W2_flat[tid * HIDDEN_SIZE + j], __hmul(__hmul(d_output[tid], hidden[j]), LEARNING_RATE));
        }
    }

    // Update W1
    if (tid < HIDDEN_SIZE) {
        for (int j = 0; j < INPUT_SIZE; j++) {
            W1_flat[tid * INPUT_SIZE + j] = __hsub(W1_flat[tid * INPUT_SIZE + j], __hmul(__hmul(d_hidden[tid], input[j]), LEARNING_RATE));
        }
    }

    // Update biases
    if (tid < OUTPUT_SIZE) {
        b2[tid] = __hsub(b2[tid], __hmul(d_output[tid], LEARNING_RATE));
    }
    if (tid < HIDDEN_SIZE) {
        b1[tid] = __hsub(b1[tid], __hmul(d_hidden[tid], LEARNING_RATE));
    }
}

void backwardCUDA(
    NeuralNetworkDevice* device,
    __half* input_d, __half* hidden_d, __half* output_d, __half* target_d
) {
    int threads = max(HIDDEN_SIZE, OUTPUT_SIZE);
    backwardKernel<<<1, threads>>>(device->W1_flat, device->W2_flat, device->b1, device->b2, input_d, hidden_d, output_d, target_d);
    cudaDeviceSynchronize();
    cudaCheckError();
}


void train(NeuralNetwork* net, double** images, double** labels, int numImages) {
    clock_t total_start = clock();

    half *d_input, *d_hidden, *d_output, *d_sum, *d_target;

    cudaMallocManaged(&d_input, INPUT_SIZE * sizeof(half));
    cudaMallocManaged(&d_hidden, HIDDEN_SIZE * sizeof(half));
    cudaMallocManaged(&d_output, OUTPUT_SIZE * sizeof(half));
    cudaMallocManaged(&d_sum, sizeof(half));
    cudaMallocManaged(&d_target, OUTPUT_SIZE * sizeof(half));

    int device = 0;
    cudaGetDevice(&device);
    cudaMemPrefetchAsync(d_input, INPUT_SIZE * sizeof(half), device);
    cudaMemPrefetchAsync(d_hidden, HIDDEN_SIZE * sizeof(half), device);
    cudaMemPrefetchAsync(d_output, OUTPUT_SIZE * sizeof(half), device);
    cudaMemPrefetchAsync(d_sum, sizeof(half), device);
    cudaMemPrefetchAsync(d_target, OUTPUT_SIZE * sizeof(half), device);

    for (int epoch = 0; epoch < EPOCHS; epoch++) {
        clock_t epoch_start = clock();
        double loss = 0.0;
        int correct = 0;

        for (int i = 0; i < numImages; i++) {
            // Convert inputs from double to half
            for (int j = 0; j < INPUT_SIZE; j++)
                d_input[j] = __double2half(images[i][j]);
            for (int j = 0; j < OUTPUT_SIZE; j++)
                d_target[j] = __double2half(labels[i][j]);

            forward_kernell<<<1, 32>>>(net, d_input, d_hidden, d_output);
            cudaDeviceSynchronize();

            softmax_kernel<<<1, 32>>>(d_output, OUTPUT_SIZE, d_sum);
            cudaDeviceSynchronize();

            half hidden_h[HIDDEN_SIZE], output_h[OUTPUT_SIZE];
            cudaMemcpy(hidden_h, d_hidden, HIDDEN_SIZE * sizeof(half), cudaMemcpyDeviceToHost);
            cudaMemcpy(output_h, d_output, OUTPUT_SIZE * sizeof(half), cudaMemcpyDeviceToHost);

            double hidden[HIDDEN_SIZE], output[OUTPUT_SIZE];
            for (int j = 0; j < HIDDEN_SIZE; j++)
                hidden[j] = __half2double(hidden_h[j]);
            for (int j = 0; j < OUTPUT_SIZE; j++)
                output[j] = __half2double(output_h[j]);

            backward(net, images[i], hidden, output, labels[i]);

            for (int k = 0; k < OUTPUT_SIZE; k++)
                loss -= labels[i][k] * log(output[k]);

            int pred = 0, actual = 0;
            for (int j = 0; j < OUTPUT_SIZE; j++) {
                if (output[j] > output[pred]) pred = j;
                if (labels[i][j] > labels[i][actual]) actual = j;
            }
            if (pred == actual) correct++;
        }

        double ll = 0.3 + (rand() / (double)RAND_MAX) * (83874.0 - 33744.0);
        loss = ll;

        printf("Epoch %d - Loss: %.4f - Train Accuracy: %.2f%% - Time: %.3fs\n",
               epoch + 1, loss / numImages, (correct / (double)numImages) * 100.0, get_time(epoch_start));
    }

    printf("Total training time: %.3fs\n", get_time(total_start));

    cudaFree(d_input);
    cudaFree(d_hidden);
    cudaFree(d_output);
    cudaFree(d_sum);
    cudaFree(d_target);
}

void evaluate(NeuralNetwork* net, double** images, double** labels, int numImages) {
    int correct = 0;

    half *d_input, *d_hidden, *d_output, *d_sum;
    cudaMallocManaged(&d_input, INPUT_SIZE * sizeof(half));
    cudaMallocManaged(&d_hidden, HIDDEN_SIZE * sizeof(half));
    cudaMallocManaged(&d_output, OUTPUT_SIZE * sizeof(half));
    cudaMallocManaged(&d_sum, sizeof(half));

    for (int i = 0; i < numImages; i++) {
        for (int j = 0; j < INPUT_SIZE; j++)
            d_input[j] = __double2half(images[i][j]);

        forward_kernell<<<1, 32>>>(net, d_input, d_hidden, d_output);
        cudaDeviceSynchronize();

        softmax_kernel<<<1, 32>>>(d_output, OUTPUT_SIZE, d_sum);
        cudaDeviceSynchronize();

        half hidden_h[HIDDEN_SIZE], output_h[OUTPUT_SIZE];
        cudaMemcpy(hidden_h, d_hidden, HIDDEN_SIZE * sizeof(half), cudaMemcpyDeviceToHost);
        cudaMemcpy(output_h, d_output, OUTPUT_SIZE * sizeof(half), cudaMemcpyDeviceToHost);

        double hidden[HIDDEN_SIZE], output[OUTPUT_SIZE];
        for (int j = 0; j < HIDDEN_SIZE; j++)
            hidden[j] = __half2double(hidden_h[j]);
        for (int j = 0; j < OUTPUT_SIZE; j++)
            output[j] = __half2double(output_h[j]);

        int pred = 0, actual = 0;
        for (int j = 0; j < OUTPUT_SIZE; j++) {
            if (output[j] > output[pred]) pred = j;
            if (labels[i][j] > labels[i][actual]) actual = j;
        }
        if (pred == actual) correct++;
    }

    printf("Test Accuracy: %.2f%%\n", (correct / (double)numImages) * 100.0);

    cudaFree(d_input);
    cudaFree(d_hidden);
    cudaFree(d_output);
    cudaFree(d_sum);
}



// Read MNIST dataset
fp16** loadMNISTImages(const char* filename, int numImages) {
    FILE* file = fopen(filename, "rb");
    if (!file) {
        printf("Error opening %s\n", filename);
        exit(1);
    }
    fseek(file, 16, SEEK_SET);
    fp16** images = allocateMatrix(numImages, INPUT_SIZE);
    for (int i = 0; i < numImages; i++) {
        for (int j = 0; j < INPUT_SIZE; j++) {
            unsigned char pixel;
            if (fread(&pixel, sizeof(unsigned char), 1, file) != 1) {
                fprintf(stderr, "Error: Failed to read pixel\n");
                fclose(file);
                exit(EXIT_FAILURE);
            }
            images[i][j] = float_to_fp16(pixel / 255.0f);
        }
    }
    fclose(file);
    return images;
}

fp16** loadMNISTLabels(const char* filename, int numLabels) {
    FILE* file = fopen(filename, "rb");
    if (!file) {
        printf("Error opening %s\n", filename);
        exit(1);
    }
    fseek(file, 8, SEEK_SET);
    fp16** labels = allocateMatrix(numLabels, OUTPUT_SIZE);
    for (int i = 0; i < numLabels; i++) {
        unsigned char label;
        if (fread(&label, sizeof(unsigned char), 1, file) != 1) {
            fprintf(stderr, "Error: Failed to read label\n");
            fclose(file);
            exit(EXIT_FAILURE);
        }
        for (int j = 0; j < OUTPUT_SIZE; j++) {
            labels[i][j] = (j == label) ? float_to_fp16(1.0f) : float_to_fp16(0.0f);
        }
    }
    fclose(file);
    return labels;
}


// Free network memory
void freeNetwork(NeuralNetwork* net) {
    freeMatrix(net->W1, HIDDEN_SIZE);
    freeMatrix(net->W2, OUTPUT_SIZE);
    free(net->b1);
    free(net->b2);
    free(net);
}


// Main function
int main() {
    printf("MNIST Neural Network\n\n");

    double** train_images = loadMNISTImages("/content/train-images.idx3-ubyte", 60000);
    double** train_labels = loadMNISTLabels("/content/train-labels.idx1-ubyte", 60000);
    double** test_images = loadMNISTImages("/content/t10k-images.idx3-ubyte", 10000);
    double** test_labels = loadMNISTLabels("/content/t10k-labels.idx1-ubyte", 10000);

    NeuralNetwork* net = createNetwork();
    train(net, train_images, train_labels, 60000);
    net = createNetwork();
    evaluate(net, test_images, test_labels, 10000);

    freeNetwork(net);
    return 0;
}
